In [1]:
!pip install streamlit
!pip install pyngrok
!pip install pandas
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.3 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import pandas as pd
import streamlit as st

st.title("Simple Data Dashboard")

uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)

    # Convert 'Date' column to datetime if it exists
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])

    # Rename columns to remove spaces (optional but recommended)
    df.columns = df.columns.str.replace(" ", "_")

    st.subheader("Data Preview")
    st.write(df.head())

    st.subheader("Data Summary")
    st.write(df.describe())

    st.subheader("Filter Data")
    columns = df.columns.tolist()
    selected_column = st.selectbox("Select column to filter by", columns)

    if df[selected_column].dtype == object:
        unique_values = df[selected_column].unique()
        selected_value = st.selectbox("Select value", unique_values)
        filtered_df = df[df[selected_column] == selected_value]
    else:
        min_value, max_value = float(df[selected_column].min()), float(df[selected_column].max())
        selected_value = st.slider(f"Select range for {selected_column}", min_value, max_value, (min_value, max_value))
        filtered_df = df[(df[selected_column] >= selected_value[0]) & (df[selected_column] <= selected_value[1])]

    st.write(filtered_df)

    st.subheader("Plot Data")

    # Only show numeric columns for plotting
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    x_column = st.selectbox("Select x-axis column", df.columns.tolist())  # Allow Date selection too
    y_column = st.selectbox("Select y-axis column", numeric_columns)

    if st.button("Generate Plot"):
        if not filtered_df.empty:
            st.line_chart(filtered_df.set_index(x_column)[y_column])
        else:
            st.error("Filtered data is empty! Try adjusting the filters.")
else:
    st.write("Waiting on file upload...")


Writing app.py


In [3]:
!pip install pyngrok
from pyngrok import ngrok

# 1. Sign up for ngrok and get your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
# 2. Set your authtoken in your code
ngrok.set_auth_token("2tJafLdInDvZWHmOoyvuaSMeJmI_4DvTaGjMdniUwqNjtBcyu")

# Start Streamlit
!streamlit run app.py &>/dev/null &

# Expose port 8501
public_url = ngrok.connect(8501).public_url
print(f"Public URL: {public_url}")

Public URL: https://4cc6-34-105-101-42.ngrok-free.app
